In [1]:
import numpy as np
import pandas as pd
import scipy.sparse as sp
import FileManager

In [3]:
ratings = FileManager.load_data()
users_to_recommend = np.array(FileManager.load_target()).squeeze()
urm_all,urm_train,urm_validation= FileManager.split_data(ratings)
urm_train_validation = urm_train + urm_validation

> Importing file...
> Importing file... Completed!
> Importing file...
> Importing file... Completed!


In [20]:
sp.save_npz("saved_urm/urm_train.npz", urm_train, compressed=True)
sp.save_npz("saved_urm/urm_validation.npz", urm_validation, compressed=True)
sp.save_npz("saved_urm/urm_train_validation.npz", urm_train_validation, compressed=True)

In [2]:
urm_train_l=sp.load_npz("saved_urm/urm_train.npz")
urm_validation_l=sp.load_npz("saved_urm/urm_validation.npz")
urm_train_validation_l=sp.load_npz("saved_urm/urm_train_validation.npz")

In [6]:
from Evaluation.Evaluator import EvaluatorHoldout

evaluator_validation = EvaluatorHoldout(urm_validation_l, cutoff_list=[10])

EvaluatorHoldout: Ignoring 13646 ( 0.0%) Users that have less than 1 test interactions


In [11]:
from Recommenders.Recommender_import_list import *

from HyperparameterTuning.run_hyperparameter_search import runHyperparameterSearch_Collaborative
import traceback

import os, multiprocessing
from functools import partial

collaborative_algorithm_list = [
        PureSVDRecommender
    ]

output_folder_path = "Optimization/"


from Evaluation.Evaluator import EvaluatorHoldout

cutoff_list = [5,10,20]
metric_to_optimize = "MAP"
cutoff_to_optimize = 10

n_cases = 100
n_random_starts = int(n_cases/3)


runParameterSearch_Collaborative_partial = partial(runHyperparameterSearch_Collaborative,
                                                    URM_train = urm_train_l,
                                                    metric_to_optimize = metric_to_optimize,
                                                    cutoff_to_optimize = cutoff_to_optimize,
                                                    n_cases = n_cases,
                                                    n_random_starts = n_random_starts,
                                                    evaluator_validation_earlystopping = evaluator_validation,
                                                    evaluator_validation = evaluator_validation,
                                                    output_folder_path = output_folder_path,
                                                    resume_from_saved = True,
                                                    #allow_bias_URM=True,
                                                    similarity_type_list = ["cosine"],
                                                    parallelizeKNN = False,
                                                    ) 





pool = multiprocessing.Pool(processes=int(multiprocessing.cpu_count()), maxtasksperchild=1)
pool.map(runParameterSearch_Collaborative_partial, collaborative_algorithm_list)

Tensorflow is not available
SearchBayesianSkopt: Extending previous number of cases from 80 to 100.

SearchBayesianSkopt: Resuming 'PureSVDRecommender'... Loaded 80 configurations.
Iteration No: 1 started. Evaluating function at random point.
Iteration No: 1 ended. Evaluation done at random point.
Time taken: 1.9351
Function value obtained: -0.2335
Current minimum: -0.2335
Iteration No: 2 started. Evaluating function at random point.
SearchBayesianSkopt: Testing config: {'num_factors': 87}
PureSVDRecommender: Computing SVD decomposition...
PureSVDRecommender: Computing SVD decomposition... done in 6.63 sec
EvaluatorHoldout: Processed 13646 (100.0%) in 13.29 sec. Users per second: 1027
SearchBayesianSkopt: Config 80 is suboptimal. Config: {'num_factors': 87} - results: PRECISION: 0.3394181, PRECISION_RECALL_MIN_DEN: 0.3408638, RECALL: 0.0585425, MAP: 0.1993070, MAP_MIN_DEN: 0.1998830, MRR: 0.5931145, NDCG: 0.3538999, F1: 0.0998611, HIT_RATE: 0.9504617, ARHR_ALL_HITS: 1.0710643, NOVELTY:

[None]

In [5]:
from Recommenders.ScoresHybridRecommender import ScoresHybridRecommender
from Recommenders.ScoresHybrid2Recommender import ScoresHybrid2Recommender
from Recommenders.ScoresHybrid3Recommender import ScoresHybrid3Recommender
from Recommenders.SLIM.SLIMElasticNetRecommender import SLIMElasticNetRecommender
from Recommenders.MatrixFactorization.PureSVDRecommender import PureSVDRecommender
from Recommenders.KNN.UserKNNCFRecommender import UserKNNCFRecommender

################ LOAD SLIM ################
slimElasticNetLoaded= SLIMElasticNetRecommender(urm_train)
slimElasticNetLoaded.load_model(folder_path='saved_models/train/',file_name='slimElasticNet_best')
################ LOAD PURE SVD ################
pureSVDloaded= PureSVDRecommender(urm_train)
pureSVDloaded.load_model(folder_path='saved_models/train/',file_name='pureSVD')

SLIMElasticNetRecommender: Loading model from file 'saved_models/train/slimElasticNet_best'
SLIMElasticNetRecommender: Loading complete
PureSVDRecommender: Loading model from file 'saved_models/train/pureSVD'
PureSVDRecommender: Loading complete


In [13]:
pureSVD= PureSVDRecommender(urm_train)
pureSVD.fit(num_factors=37)

PureSVDRecommender: Computing SVD decomposition...
PureSVDRecommender: Computing SVD decomposition... done in 3.82 sec


In [14]:
scoreshybridrecommender= ScoresHybridRecommender(urm_train,slimElasticNetLoaded,pureSVD)
scoreshybridrecommender.fit(alpha=0.77535)

In [15]:
result_df, _ = evaluator_validation.evaluateRecommender(scoreshybridrecommender)
print(result_df['MAP'])

EvaluatorHoldout: Ignoring 13646 ( 0.0%) Users that have less than 1 test interactions
EvaluatorHoldout: Processed 13646 (100.0%) in 42.38 sec. Users per second: 322
cutoff
10    0.245065
Name: MAP, dtype: object
